In [1]:
import pandas as pd
import pickle
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.conferences import Conferences
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random as rnd
from utils import  score_table
import importlib

#import argparse

import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Predictive, Trace_ELBO
from pyro.optim import ClippedAdam
from torch.distributions import constraints

In [8]:
majorConfs = ["big-12","Big 12",
              "big-10","Big Ten","big-ten",
              "sec","SEC",
              "big-east","Big East",
              "pac-12","Pac-12",
              "acc","ACC",
              "wcc", "WCC",
              "atlantic-10","Atlantic 10"]
confNameNorm = {"Big 12":"big-12",
              "Big Ten":"big-ten",
              "big-ten":"big-ten",
              "SEC":"sec",
              "Big East":"big-east",
              "Pac-12":"pac-12",
              "ACC":"acc",
              "WCC":"wcc",
               "Atlantic 10":"atlantic-10"}
def getDataForTeams(teams) :
    dataset = pd.DataFrame()
    for team in teams:
        if team.conference in majorConfs:
            tgDF = team.schedule.dataframe
            tgDF['team_name'] = team.name
            tgDF['team_conference'] = team.conference
            dataset = pd.concat([dataset,tgDF])     
    return dataset


In [3]:
#execute this cell to grab the data from the site
season_team_list = Teams("2022")
df = getDataForTeams(season_team_list)
compression_opts = dict(method='zip',
                        archive_name='outv3.csv')  
df.to_csv('data/bigconfs2022Season.zip', compression=compression_opts) 
pickle.dump( season_team_list, open( "data/teams2022.p", "wb" ) )

In [9]:
#execute this cell to just load the local files
season_team_list = pickle.load( open( "data/teams2022.p", "rb" ) )
c_opts = dict(method='zip')
df = pd.read_csv('data/bigconfs2022Season.zip',compression='zip')

In [10]:
bdf = df
#only have games between big conferences
bdf = bdf[bdf["opponent_conference"].isin(majorConfs)]
#get rid of duplicates
bdf.drop_duplicates(subset='boxscore_index', keep="first")
#drop Neutral games for now
#bdf = bdf[bdf["location"].isin(["Home","Away"])]
#create Home, Away, neutral indicator columns
#bdf = pd.get_dummies(bdf,prefix=['location'], columns = ['location'])

bdf["opponent_conference"].replace(confNameNorm, inplace=True)

bdf.reset_index(inplace=True, drop=True)

bdf.head()

/usr/local/lib/python3.9/site-packages/pandas/core/series.py:4576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Unnamed: 0,arena,boxscore_index,date,datetime,game,location,opponent_abbr,opponent_conference,opponent_name,...,points_against,points_for,result,season_losses,season_wins,streak,time,type,team_name,team_conference
0,2021-11-28-18-alabama,HP Field House,2021-11-28-18-alabama,"Sun, Nov 28, 2021",2021-11-28 18:30:00,7,Neutral,miami-fl,acc,Miami (FL),...,64.0,96.0,Win,1.0,6.0,W 2,6:30p,Reg,Alabama,sec
1,2021-12-04-20-gonzaga,KeyArena,2021-12-04-20-gonzaga,"Sat, Dec 4, 2021",2021-12-04 20:00:00,8,Neutral,gonzaga,wcc,Gonzaga,...,82.0,91.0,Win,1.0,7.0,W 3,8:00p,Reg,Alabama,sec
2,2021-12-29-21-alabama,Coleman Coliseum,2021-12-29-21-alabama,"Wed, Dec 29, 2021",2021-12-29 21:00:00,13,Home,tennessee,sec,Tennessee,...,68.0,73.0,Win,3.0,10.0,W 1,9:00p,Reg,Alabama,sec
3,2022-01-05-19-florida,Stephen C. O'Connell Center,2022-01-05-19-florida,"Wed, Jan 5, 2022",2022-01-05 19:00:00,14,Away,florida,sec,Florida,...,70.0,83.0,Win,3.0,11.0,W 2,7:00p,Reg,Alabama,sec
4,2022-01-08-15-missouri,Mizzou Arena,2022-01-08-15-missouri,"Sat, Jan 8, 2022",2022-01-08 15:30:00,15,Away,missouri,sec,Missouri,...,92.0,86.0,Loss,4.0,11.0,L 1,3:30p,Reg,Alabama,sec


In [11]:
bdf.columns

Index(['Unnamed: 0', 'arena', 'boxscore_index', 'date', 'datetime', 'game',
       'location', 'opponent_abbr', 'opponent_conference', 'opponent_name',
       'opponent_rank', 'overtimes', 'points_against', 'points_for', 'result',
       'season_losses', 'season_wins', 'streak', 'time', 'type', 'team_name',
       'team_conference'],
      dtype='object')

In [19]:
bsdf = bdf[['date','team_name','team_conference','opponent_name',
     'opponent_conference','points_against', 'points_for', 'location']]



In [16]:
bdf['team_name'].unique().tolist()

['Alabama',
 'Arizona State',
 'Arizona',
 'Arkansas',
 'Auburn',
 'Baylor',
 'Boston College',
 'Brigham Young',
 'Butler',
 'California',
 'Clemson',
 'Colorado',
 'Connecticut',
 'Creighton',
 'Davidson',
 'Dayton',
 'DePaul',
 'Duke',
 'Duquesne',
 'Florida State',
 'Florida',
 'Fordham',
 'George Mason',
 'George Washington',
 'Georgetown',
 'Georgia Tech',
 'Georgia',
 'Gonzaga',
 'Illinois',
 'Indiana',
 'Iowa State',
 'Iowa',
 'Kansas State',
 'Kansas',
 'Kentucky',
 'La Salle',
 'Louisiana State',
 'Louisville',
 'Loyola Marymount',
 'Marquette',
 'Maryland',
 'Massachusetts',
 'Miami (FL)',
 'Michigan State',
 'Michigan',
 'Minnesota',
 'Mississippi State',
 'Mississippi',
 'Missouri',
 'Nebraska',
 'NC State',
 'North Carolina',
 'Northwestern',
 'Notre Dame',
 'Ohio State',
 'Oklahoma State',
 'Oklahoma',
 'Oregon State',
 'Oregon',
 'Pacific',
 'Penn State',
 'Pepperdine',
 'Pittsburgh',
 'Portland',
 'Providence',
 'Purdue',
 'Rhode Island',
 'Richmond',
 'Rutgers',
 "Sai

In [17]:
def transformData(data):
    pl_data = data

    ng = len(pl_data)  # number of games
    #npr = 45  # predict the last 5 rounds of games
    ngob = ng  # number of games to train

    teams = list(set(pl_data['team_name'].unique().tolist()).union(set(pl_data['opponent_name'].unique().tolist())))
    teams = pd.DataFrame(teams, columns=["Team"])
    teams["i"] = teams.index

    df = pd.merge(pl_data, teams, left_on="team_name", right_on="Team", how="left")
    df = df.rename(columns={"i": "team_id"}).drop("Team", axis=1)
    df = pd.merge(df, teams, left_on="opponent_name", right_on="Team", how="left")
    df = df.rename(columns={"i": "opponent_id"}).drop("Team", axis=1)

    print(df.head())

    return teams, df


In [29]:
tt,dd = transformData(bsdf)

                date team_name team_conference opponent_name  \
0  Sun, Nov 28, 2021   Alabama             sec    Miami (FL)   
1   Sat, Dec 4, 2021   Alabama             sec       Gonzaga   
2  Wed, Dec 29, 2021   Alabama             sec     Tennessee   
3   Wed, Jan 5, 2022   Alabama             sec       Florida   
4   Sat, Jan 8, 2022   Alabama             sec      Missouri   

  opponent_conference  points_against  points_for location  team_id  \
0                 acc            64.0        96.0  Neutral       13   
1                 wcc            82.0        91.0  Neutral       13   
2                 sec            68.0        73.0     Home       13   
3                 sec            70.0        83.0     Away       13   
4                 sec            92.0        86.0     Away       13   

   opponent_id  
0           31  
1           59  
2           75  
3           37  
4           63  


In [31]:
dd.points_for.describe()

count    2107.000000
mean       70.339345
std        11.389553
min        29.000000
25%        63.000000
50%        70.000000
75%        77.000000
max       117.000000
Name: points_for, dtype: float64

In [36]:

class FoldedTransform(dist.transforms.AbsTransform):
    def log_abs_det_jacobian(self, x, y):
        return torch.zeros_like(x)

def model(team1_id, team2_id, score1_obs=None, score2_obs=None):
    # hyperpriors
    alpha  = pyro.sample("alpha", dist.Normal(1.0, 3.0))
    sd_att = pyro.sample("sd_att", dist.TransformedDistribution(dist.StudentT(3.0, 0.0, 4), FoldedTransform()))
    sd_def = pyro.sample("sd_def", dist.TransformedDistribution(dist.StudentT(3.0, 0.0, 4), FoldedTransform()))
    #student t: degrees-of-freedom, mean, width
    #started with width = 2.5

    nt = len(np.unique(team1_id))

    # team-specific model parameters
    with pyro.plate("plate_teams", nt):
        attack = pyro.sample("attack", dist.Normal(0, sd_att))
        defend = pyro.sample("defend", dist.Normal(0, sd_def))

    # likelihood
    theta1 = torch.exp(alpha + attack[team1_id] - defend[team2_id])
    theta2 = torch.exp(alpha + attack[team2_id] - defend[team1_id])

    with pyro.plate("data", len(team1_id)):
        pyro.sample("s1", dist.Poisson(theta1), obs=score1_obs)
        pyro.sample("s2", dist.Poisson(theta2), obs=score2_obs)

def guide(team1_id, team2_id, score1_obs=None, score2_obs=None):
    mu_locs = pyro.param("mu_loc", torch.tensor(0.0).expand(3))
    mu_scales = pyro.param("mu_scale", torch.tensor(0.1).expand(3), constraint=constraints.positive)

    pyro.sample("alpha", dist.Normal(mu_locs[0], mu_scales[0]))
    pyro.sample("sd_att", dist.LogNormal(mu_locs[1], mu_scales[1]))
    pyro.sample("sd_def", dist.LogNormal(mu_locs[2], mu_scales[2]))

    nt = len(np.unique(team1_id))

    mu_team_locs = pyro.param("mu_team_loc", torch.tensor(0.0).expand(2, nt))
    mu_team_scales = pyro.param("mu_team_scale", torch.tensor(0.1).expand(2, nt), constraint=constraints.positive)

    with pyro.plate("plate_teams", nt):
        pyro.sample("attack", dist.Normal(mu_team_locs[0], mu_team_scales[0]))
        pyro.sample("defend", dist.Normal(mu_team_locs[1], mu_team_scales[1]))

def run_inference(model, guide, team1_id, team2_id, score1, score2, args):
    gamma = 0.01  # final learning rate will be gamma * initial_lr
    lrd = gamma ** (1 / args.num_iterations)

    svi = SVI(
        model=model,
        guide=guide,
        optim=ClippedAdam({"lr": args.learning_rate, "lrd": lrd}),
        loss=Trace_ELBO(num_particles=args.num_particles),
    )

    pyro.clear_param_store()  # clear global parameter cache
    pyro.set_rng_seed(args.rng_seed)

    advi_loss = []
    for j in range(args.num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(
            team1_id=team1_id,
            team2_id=team2_id,
            score1_obs=score1.float(),
            score2_obs=score2.float(),
        )
        advi_loss.append(loss)
        if j % 100 == 0:
            print("[iteration %4d] loss: %.4f" % (j + 1, loss))

    print("Posterior: ")
    for i in pyro.get_param_store().items():
        print(i)

    fit = Predictive(model=model, guide=guide, num_samples=2000)(
        home_id=home_id, away_id=away_id
    )

    return fit

def runThePyroFit(data,args):
    print("transforming data...")
    teams, df =  transformData(data)
    train = df

    print("Starting inference...")
    fit = run_inference(
        model,
        guide,
        torch.tensor(train["team_id"]),
        torch.tensor(train["opponent_id"]),
        torch.tensor(train["points_for"]),
        torch.tensor(train["points_against"]),
        args,
    )
    return fit,teams,df

#def runThePyroCodeRest(data,args,fit):
def runThePyroCodeRest(fit,teams,df):


    print("Analyse posterior...")


    # Predicted goals and table
    predict = df

    predictive = Predictive(
        model=model, guide=guide, num_samples=2000, return_sites=["s1", "s2"]
    )
 
    predicted_score = predictive(
        home_id=predict["team_id"].values,
        away_id=predict["opponent_id"].values,
    )

    predicted_full = predict.copy()
    predicted_full = predicted_full.assign(
        score1=predicted_score["s1"].mean(axis=0).round(),
        score1error=predicted_score["s1"].std(axis=0),
        score2=predicted_score["s2"].mean(axis=0).round(),
        score2error=predicted_score["s2"].std(axis=0),
    )

    predicted_full = train.append(
        predicted_full.drop(columns=["score1error", "score2error"])
    )

    print(score_table(df))
    print(score_table(predicted_full))

def qualityCalc(fit,teams,df,train):
    # Attack and defence
    quality = teams.copy()

    quality = quality.assign(
        attack=fit["attack"].mean(axis=0),
        attacksd=fit["attack"].std(axis=0),
        defend=fit["defend"].mean(axis=0),
        defendsd=fit["defend"].std(axis=0),
    )
    quality = quality.assign(
        attack_low=quality["attack"] - quality["attacksd"],
        attack_high=quality["attack"] + quality["attacksd"],
        defend_low=quality["defend"] - quality["defendsd"],
        defend_high=quality["defend"] + quality["defendsd"],
    )
    #gotta debug this
    return quality


In [37]:
class Args():
    learning_rate = 0.1
    num_particles = 1
    rng_seed = 1
    num_iterations = 2000

args=Args()
fit,teams,df = runThePyroFit(bsdf,args)

transforming data...
                date team_name team_conference opponent_name  \
0  Sun, Nov 28, 2021   Alabama             sec    Miami (FL)   
1   Sat, Dec 4, 2021   Alabama             sec       Gonzaga   
2  Wed, Dec 29, 2021   Alabama             sec     Tennessee   
3   Wed, Jan 5, 2022   Alabama             sec       Florida   
4   Sat, Jan 8, 2022   Alabama             sec      Missouri   

  opponent_conference  points_against  points_for location  team_id  \
0                 acc            64.0        96.0  Neutral       13   
1                 wcc            82.0        91.0  Neutral       13   
2                 sec            68.0        73.0     Home       13   
3                 sec            70.0        83.0     Away       13   
4                 sec            92.0        86.0     Away       13   

   opponent_id  
0           31  
1           59  
2           75  
3           37  
4           63  
Starting inference...


ValueError: Error while computing log_prob at site 's1':
Expected value argument (Tensor of shape (2126,)) to be within the support (IntegerGreaterThan(lower_bound=0)) of the distribution Poisson(rate: torch.Size([2126])), but found invalid values:
tensor([96., 91., 73.,  ..., 66., 97., 82.])
Trace Shapes:       
 Param Sites:       
Sample Sites:       
   alpha dist      |
        value      |
     log_prob      |
  sd_att dist      |
        value      |
     log_prob      |
  sd_def dist      |
        value      |
     log_prob      |
  attack dist  100 |
        value  100 |
     log_prob  100 |
  defend dist  100 |
        value  100 |
     log_prob  100 |
      s1 dist 2126 |
        value 2126 |